In [ ]:
import lineapy

## Phase: Data exploration

As my initial task I'm tasked with exploring how ML could complement our existing business, specifically by predicting the eventual sales price. We have some loyal clients who want to sell with us, a more local shop, than Zillow---we have just relied on realtor known how, but the new COVID trends really threw us off track and we are seeing if the models would pick up the signals more than we can.

My immediate goal is to build a model to predic the sales price. I'm going to check out the data and see if there are some patterns for the algorithm to leverage.

In [ ]:
import pandas as pd
training_data = pd.read_csv("data/ames_train_cleaned.csv")

In [ ]:
training_data.head(10)

In [ ]:
# NBVAL_SKIP
lineapy.visualize()

Before I plot anything, I want to get a sense of the null values

In [ ]:
training_data['Lot_Area'].isnull().sum()

In [ ]:
training_data['MS_Zoning'].isnull().sum()

In [ ]:
len(training_data)

We can drop "Pool_QC" and Misc_Feature" since they are mostly nulls

In [ ]:
cleaned_data = training_data.drop(['Pool_QC', 'Misc_Feature'], axis=1)

In [ ]:
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

My sense is that the Living Area is a big predictor for sale price, maybe followed by the Garage Area.

In [ ]:
sns.jointplot(
    x='Gr_Liv_Area', 
    y='SalePrice', 
    data=cleaned_data,
    kind="reg",
    ratio=4,
    space=0,
    scatter_kws={
        's': 3,
        'alpha': 0.25
    },
    line_kws={
        'color': 'black'
    }
);

I see that the trend starts to dilute, so I want to remove the larger values for now.

In [ ]:
# NBVAL_IGNORE_OUTPUT
cleaned_data = cleaned_data[cleaned_data['Gr_Liv_Area']  < 3000]
cleaned_data

In [ ]:
print("Number of records removed:", len(training_data) - len(cleaned_data))

In [ ]:
sns.jointplot(
    x='Garage_Area', 
    y='SalePrice', 
    data=cleaned_data,
    kind="reg",
    ratio=4,
    space=0,
    scatter_kws={
        's': 3,
        'alpha': 0.25
    },
    line_kws={
        'color': 'black'
    }
);

In [ ]:
# NBVAL_IGNORE_OUTPUT
cleaned_data = cleaned_data[cleaned_data['Garage_Area']  < 1250]
cleaned_data

In [ ]:
print("Number of records removed:", len(training_data) - len(cleaned_data))

In [ ]:
lineapy.visualize()

In [ ]:
fig, axs = plt.subplots(nrows=2)

sns.boxplot(
    x='Neighborhood',
    y='SalePrice',
    data=cleaned_data.sort_values('Neighborhood'),
    ax=axs[0]
)

sns.countplot(
    x='Neighborhood',
    data=cleaned_data.sort_values('Neighborhood'),
    ax=axs[1]
)

# Draw median price
axs[0].axhline(
    y=cleaned_data['SalePrice'].median(), 
    color='red',
    linestyle='dotted'
)

# Label the bars with counts
for patch in axs[1].patches:
    x = patch.get_bbox().get_points()[:, 0]
    y = patch.get_bbox().get_points()[1, 1]
    axs[1].annotate(f'{int(y)}', (x.mean(), y), ha='center', va='bottom')
    
# Format x-axes
axs[1].set_xticklabels(axs[1].xaxis.get_majorticklabels(), rotation=90)
axs[0].xaxis.set_visible(False)

# Narrow the gap between the plots
plt.subplots_adjust(hspace=0.01)

I verify that Neighborhood is a very promising way to go forward.

However, in order to be able to also use linear models, I must map these neighborhoods back into numeric values (one hot encoding).

In [ ]:
from pandas.api.types import CategoricalDtype

from sklearn.feature_extraction import DictVectorizer

vec_enc = DictVectorizer()
vec_enc.fit(cleaned_data[['Neighborhood']].to_dict(orient='records'))
Neighborhood_data = vec_enc.transform(cleaned_data[['Neighborhood']].to_dict(orient='records')).toarray()
Neighborhood_cats = vec_enc.get_feature_names()
Neighborhood = pd.DataFrame(Neighborhood_data, columns=Neighborhood_cats)
cleaned_data = pd.concat([cleaned_data, Neighborhood], axis=1)
cleaned_data = cleaned_data.drop(columns=Neighborhood_cats[0])

In sum, the columns that I should focus on is:

- Gr_Liv_Area
- Garage_Area
- regex='Neightborhood'

In [ ]:
!mkdir -p outputs
cleaned_data.filter(
    regex="Neighborhood=.|Gr_Liv_Area|Garage_Area|SalePrice"
).to_csv("outputs/cleaned_data_housing.csv", index=False)

## Task 1: Pipeline

I want to re-run this when I have new data---i.e. when `ames_train_cleaned.csv` updates. However, I realize that I have to go to the `Exploration_without_Linea.ipynb` file to extract the process. Try to lift it manually into a airflow job.

In [ ]:
# NBVAL_SKIP
# clear airflow dags to demonstrate that we are creating these dags
!rm /tmp/airflow_home/dags/*.py

In [ ]:
artifact = lineapy.save(lineapy.file_system, "cleaned_data_housing")

In [ ]:
# NBVAL_SKIP
artifact.visualize()

In [ ]:
# NBVAL_SKIP
!rm outputs/cleaned_data_housing.csv

In [ ]:
artifact.to_airflow();

In [ ]:
print(artifact.code)

If we wait for airflow to run, we can see that the CSV is now re-computed:

In [ ]:
# NBVAL_IGNORE_OUTPUT
!head -n 10 outputs/cleaned_data_housing.csv